In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import pandas as pd

# ------------------- Semillas para reproducibilidad -------------------
SEMEJANTE_SEED = 42
np.random.seed(SEMEJANTE_SEED)
random.seed(SEMEJANTE_SEED)

# ------------------- Carga de datos y partición Train/Test -------------------
datos = load_breast_cancer()
X = datos.data
y = datos.target
nombres_caracteristicas = np.array(datos.feature_names)

# Limitar el conjunto de características a 6
X = X[:, :6]
nombres_caracteristicas = nombres_caracteristicas[:6]

# Mostrar la lista de características usadas
print("Características (6) utilizadas en el experimento:")
for i, nm in enumerate(nombres_caracteristicas):
    print(f"  [{i}] {nm}")
print("-" * 60)

# Dividir en train y test
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=SEMEJANTE_SEED
)

n_caracteristicas = X_entrenamiento.shape[1]  # Ahora 6 características

# --------- helpers para mapear máscara <-> nombres de características ---------
def obtener_caracteristicas_de_mascara(mascara: np.ndarray) -> list[str]:
    """Devuelve los nombres de las características activas (1s) en la máscara."""
    return nombres_caracteristicas[np.where(mascara == 1)[0]].tolist()

def mascara_legible(mascara: np.ndarray) -> str:
    """Cadena con bits y nombres activos."""
    activos = obtener_caracteristicas_de_mascara(mascara)
    return f"{mascara.tolist()}  ->  activas: {activos if activos else 'NINGUNA'}"

# ------------------- Hiperparámetros del Algoritmo Genético -------------------
TAM_POBLACION = 12
N_GENERACIONES = 5
TAM_TORNEO = 3
PROB_CRUCE = 0.9
PROB_MUTACION = 0.02
ELITISMO = True
ALFA_ESPARCIDAD = 0.02   # penalización por usar muchas características

# ------------------- Función de aptitud -------------------
def evaluar_cromosoma(mascara: np.ndarray) -> float:
    """
    Fitness:
    - 0 si no selecciona nada.
    - promedio de accuracy (5-fold CV) de LR estandarizada.
    - penalización por cantidad de características para favorecer parsimonia.
    """
    if mascara.sum() == 0:
        return 0.0

    X_sub = X_entrenamiento[:, mascara == 1]

    pipe = Pipeline([
        ("escalador", StandardScaler()),
        ("clasificador", LogisticRegression(max_iter=500, solver="liblinear", random_state=SEMEJANTE_SEED)),
    ])

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEMEJANTE_SEED)
    puntuaciones = cross_val_score(pipe, X_sub, y_entrenamiento, cv=cv, scoring="accuracy")
    acc = puntuaciones.mean()

    penalizacion_espacidad = ALFA_ESPARCIDAD * (mascara.sum() / n_caracteristicas)
    return max(acc - penalizacion_espacidad, 0.0)

# ------------------- Operadores del AG -------------------
def inicializar_poblacion(tam_poblacion: int, n_feats: int) -> np.ndarray:
    """Población binaria, prob. de 1 ~ 0.5; forzar al menos 1 activa."""
    poblacion = np.zeros((tam_poblacion, n_feats), dtype=int)
    for i in range(tam_poblacion):
        mascara = (np.random.rand(n_feats) < 0.5).astype(int)
        if mascara.sum() == 0:
            mascara[np.random.randint(0, n_feats)] = 1
        poblacion[i] = mascara
    return poblacion

def seleccion_por_torneo(poblacion: np.ndarray, aptitudes: np.ndarray, k: int) -> np.ndarray:
    """Torneo de tamaño k; retorna copia del mejor."""
    idxs = np.random.choice(len(poblacion), size=k, replace=False)
    mejor_idx = idxs[np.argmax(aptitudes[idxs])]
    return poblacion[mejor_idx].copy()

def cruce_por_punto(p1: np.ndarray, p2: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Cruce de un punto.
    Además de los bits, imprime los NOMBRES de las características activas
    en padres e hijos, y el punto de cruce.
    """
    if np.random.rand() > PROB_CRUCE or len(p1) < 2:
        # sin cruce: aún mostramos info de padres
        print("SIN CRUCE (probabilidad no superada):")
        print(f"  Padre 1: {mascara_legible(p1)}")
        print(f"  Padre 2: {mascara_legible(p2)}")
        print("-" * 60)
        return p1.copy(), p2.copy()

    punto = np.random.randint(1, len(p1))
    c1 = np.concatenate([p1[:punto], p2[punto:]])
    c2 = np.concatenate([p2[:punto], p1[punto:]])

    print("CRUCE DE UN PUNTO")
    print(f"  Punto de cruce: {punto}")
    print(f"  Padre 1: {mascara_legible(p1)}")
    print(f"  Padre 2: {mascara_legible(p2)}")
    print(f"  Hijo 1:  {mascara_legible(c1)}")
    print(f"  Hijo 2:  {mascara_legible(c2)}")
    print("-" * 60)

    return c1, c2

def mutar(mascara: np.ndarray) -> np.ndarray:
    """
    Mutación por bit flip.
    Imprime cada mutación con la posición y el nombre de la característica.
    """
    antes = mascara.copy()
    mutado_alguna = False
    for i in range(len(mascara)):
        if np.random.rand() < PROB_MUTACION:
            mutado_alguna = True
            bit_anterior = mascara[i]
            mascara[i] = 1 - mascara[i]
            nombre = nombres_caracteristicas[i]
            print(f"Mutación en pos {i} ({nombre}): {antes.tolist()} -> {mascara.tolist()}  (bit {bit_anterior}->{mascara[i]})")
            antes = mascara.copy()  # actualizar para mostrar cadena progresiva

    # Evitar individuos vacíos
    if mascara.sum() == 0:
        i = np.random.randint(0, len(mascara))
        mascara[i] = 1
        print(f"Corrección por vacío: activamos pos {i} ({nombres_caracteristicas[i]}). Nuevo: {mascara.tolist()}")

    if mutado_alguna:
        print(f"  Resultado tras mutación: {mascara_legible(mascara)}")
        print("-" * 60)

    return mascara

# ------------------- Bucle principal del AG -------------------
poblacion = inicializar_poblacion(TAM_POBLACION, n_caracteristicas)
aptitudes = np.array([evaluar_cromosoma(ind) for ind in poblacion])

historial_mejor = []
historial_promedio = []
mejor_cromosoma = poblacion[np.argmax(aptitudes)].copy()
mejor_aptitud = aptitudes.max()

for generacion in range(N_GENERACIONES):
    print(f"\n=== Generación {generacion+1}/{N_GENERACIONES} ===")
    nueva_poblacion = []
    if ELITISMO:
        elite = poblacion[np.argmax(aptitudes)].copy()
        print("Elitismo -> Conservando mejor individuo:", mascara_legible(elite))
        nueva_poblacion.append(elite)

    while len(nueva_poblacion) < TAM_POBLACION:
        p1 = seleccion_por_torneo(poblacion, aptitudes, TAM_TORNEO)
        p2 = seleccion_por_torneo(poblacion, aptitudes, TAM_TORNEO)

        c1, c2 = cruce_por_punto(p1, p2)
        c1 = mutar(c1)
        c2 = mutar(c2)

        nueva_poblacion.append(c1)
        if len(nueva_poblacion) < TAM_POBLACION:
            nueva_poblacion.append(c2)

    poblacion = np.array(nueva_poblacion, dtype=int)
    aptitudes = np.array([evaluar_cromosoma(ind) for ind in poblacion])

    cur_best_idx = np.argmax(aptitudes)
    cur_best_fit = aptitudes[cur_best_idx]
    if cur_best_fit > mejor_aptitud:
        mejor_aptitud = cur_best_fit
        mejor_cromosoma = poblacion[cur_best_idx].copy()

    print(f"Mejor de la generación: {mascara_legible(poblacion[cur_best_idx])}  |  fitness: {cur_best_fit:.4f}")

    historial_mejor.append(aptitudes.max())
    historial_promedio.append(aptitudes.mean())

# ------------------- Evaluación final en Test -------------------
selected_mask = mejor_cromosoma
selected_features = nombres_caracteristicas[selected_mask == 1]
n_selected = selected_mask.sum()

print("\n=== Mejor cromosoma encontrado ===")
print(mascara_legible(selected_mask))
print(f"Total de características seleccionadas: {int(n_selected)}")

pipe_final = Pipeline([
    ("escalador", StandardScaler()),
    ("clasificador", LogisticRegression(max_iter=1000, solver="liblinear", random_state=SEMEJANTE_SEED)),
])
pipe_final.fit(X_entrenamiento[:, selected_mask == 1], y_entrenamiento)

y_proba = pipe_final.predict_proba(X_prueba[:, selected_mask == 1])[:, 1]
y_pred = (y_proba >= 0.5).astype(int)

acc = accuracy_score(y_prueba, y_pred)
prec = precision_score(y_prueba, y_pred)
rec = recall_score(y_prueba, y_pred)
f1 = f1_score(y_prueba, y_pred)
roc_auc = roc_auc_score(y_prueba, y_proba)

metrics_df = pd.DataFrame({
    "Metric": ["# Selected Features", "Accuracy", "Precision", "Recall", "F1", "ROC AUC"],
    "Value": [int(n_selected), acc, prec, rec, f1, roc_auc]
})
print("\n=== Métricas en Test ===")
print(metrics_df.to_string(index=False))

# ------------------- Visualizaciones -------------------
plt.figure(figsize=(7, 4))
plt.plot(range(1, N_GENERACIONES+1), historial_mejor, label="Best fitness")
plt.plot(range(1, N_GENERACIONES+1), historial_promedio, label="Average fitness")
plt.xlabel("Generation")
plt.ylabel("Fitness (CV accuracy - sparsity penalty)")
plt.title("GA Convergence")
plt.legend()
plt.tight_layout()
plt.show()

fpr, tpr, thresholds = roc_curve(y_prueba, y_proba)
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, label=f"ROC AUC = {roc_auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Test Set)")
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()

coef = np.abs(pipe_final.named_steps["clasificador"].coef_[0])
feat_coefs = pd.Series(coef, index=selected_features).sort_values(ascending=False)

top_k = min(15, len(feat_coefs))
plt.figure(figsize=(8, 5))
feat_coefs.iloc[:top_k].plot(kind="bar")
plt.ylabel("|Coefficient| (absolute value)")
plt.title("Top Feature Weights (Logistic Regression)")
plt.tight_layout()
plt.show()

# ------------------- Tablas/archivos de salida -------------------
selected_table = pd.DataFrame({
    "Feature": selected_features,
    "Abs_Coefficient": feat_coefs[selected_features].values
}).sort_values("Abs_Coefficient", ascending=False).reset_index(drop=True)

print("\n=== Features seleccionadas (ordenadas por importancia) ===")
print(selected_table.to_string(index=False))

# Guardar a CSV (opcional)
metrics_df.to_csv("ga_feature_selection_metrics.csv", index=False)
selected_table.to_csv("ga_selected_features.csv", index=False)

print("\nArchivos guardados:")
print(" - ga_feature_selection_metrics.csv")
print(" - ga_selected_features.csv")